## Modules

In [1]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from bsoption.bsmodel import BSModel
from bsoption.nyopchain import NYopchain
Opchain = NYopchain()

from litedata import *
from litedata.loadyahoodata import getyahoodata

### US stock list setting

In [2]:
from configparser import ConfigParser
configchain = ConfigParser()
configchain.read('setting_yahoochain.ini')
chainpath = configchain['paths'].get('chainpath')

configassets = configchain['assets']
etflist = list(configassets.get('etf').split(','))
chipslist = list(configassets.get('chip').split(','))
assetlist = etflist + chipslist

## Gamma Exposure (GEX)

### OHLC of all stocks

In [3]:
dfohlcall = getyahoodata(assetlist)

[*********************100%***********************]  24 of 24 completed


d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-p

### Option chain with closing price >= 0.05 with greeks

In [4]:
def loadopchain(daystr='2022-10-03', expiry='2022-10-14', asset='NVDA', bounds=(0, 1000), opbound=0.05):
    """Obtain all options of the same expiry with closing price > 0.01 with greeks."""
    # inputdict for `.loadopdata()` method and query
    inputdict = {'asset': asset,  'optype': ('C', 'P'), 
                 'strike_lowerbound': bounds[0], 'strike_upperbound': bounds[1],  
                 'startexpiry': expiry,  'endexpiry': expiry,  'starttd': daystr,  'endtd': daystr}
    dfop = Opchain.loadopdata(inputdict)
    # Filter strike price with BOTH call & put price over 1.00
    dfcall = dfop[(dfop['optype'] == 'C') & (dfop['mid'] >= opbound)]
    dfput = dfop[(dfop['optype'] == 'P') & (dfop['mid'] >= opbound)]
    strikeset = set(dfcall['strike']).intersection(set(dfput['strike']))
    dfop = dfop[dfop['strike'].isin(strikeset)]
    # Underlying close price
    spotprice = dfohlcall.loc[daystr, f'{asset}_cl']
    dfop[f'{asset}_cl'] = spotprice
    # Dummy column of `BSmodel()` object
    tdays = (datetime.strptime(expiry, '%Y-%m-%d') - datetime.strptime(daystr, '%Y-%m-%d')).days
    dfop['BS'] = dfop.apply(lambda row: BSModel(spotprice, row['strike'], tdays, row['iv'] / 100), axis=1)
    # split option dataframe into call & put and compute delta and theta separately
    dfcall = dfop[dfop['optype'] == 'C']
    dfput = dfop[dfop['optype'] == 'P']
    dfcall['delta'] = dfcall['BS'].apply(lambda x: x.cdelta)
    dfcall['theta'] = dfcall['BS'].apply(lambda x: x.ctheta)
    dfput['delta'] = dfput['BS'].apply(lambda x: x.pdelta)
    dfput['theta'] = dfput['BS'].apply(lambda x: x.ptheta)
    dfop = pd.concat([dfcall, dfput], axis=0)
    # Compute vega & gamma columns (irrelevant of call/put)
    dfop['vega'] = dfop['BS'].apply(lambda x: x.vega)
    dfop['gamma'] = dfop['BS'].apply(lambda x: x.gamma)
    # Drop dummy column
    dfop.drop('BS', axis=1, inplace=True)
    
    return dfop

In [5]:
asset0 = 'NVDA'
dfop1, info1 = Opchain.loaddayopchain(dfohlcall, tradeday='2022-08-26', expiry='2022-09-16', asset=asset0)
dfop1

..\bsoption\bsmodel.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  d_1 = (np.log(self.S / self.K) + self.T * (self.rf + 0.5 * (self.sig ** 2))) / (self.sig * (self.T ** 0.5))
..\bsoption\bsmodel.py:33: RuntimeWarning: invalid value encountered in double_scalars
  self.gamma = round(norm.pdf(self.d_1) / (self.S * self.sig * (self.T ** 0.5)), 9)
..\bsoption\nyopchain.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcall['delta'] = dfcall['BS'].apply(lambda x: x.cdelta)
..\bsoption\nyopchain.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

,c_iv,c_vol,c_oi,c_mid,c_delta,c_theta,c_vega,c_gamma,p_iv,p_vol,p_oi,p_mid,p_delta,p_theta,p_vega,p_gamma
strike,,,,,,,,,,,,,,,,
28.75,333.20,3.0,44,134.125,0.994873,-0.045735,0.005765,0.000114,275.78,1.0,525,0.060,-0.001591,-0.013181,0.002007,0.000048
31.25,338.67,4.0,15,131.775,0.992576,-0.064527,0.008002,0.000155,262.50,10.0,907,0.060,-0.001675,-0.013146,0.002103,0.000053
47.50,254.88,5.0,411,115.525,0.989776,-0.064295,0.010595,0.000273,202.34,1.0,200,0.075,-0.002739,-0.015830,0.003286,0.000107
107.50,94.53,22.0,222,55.500,0.973630,-0.053631,0.023828,0.001658,69.92,10.0,1325,0.050,-0.005395,-0.010043,0.006033,0.000568
110.00,110.64,22.0,362,52.575,0.945665,-0.113164,0.042958,0.002554,67.97,294.0,6150,0.060,-0.006628,-0.011717,0.007240,0.000701
112.50,62.89,7.0,408,50.150,0.994052,-0.009850,0.006578,0.000688,66.80,35.0,4387,0.080,-0.008725,-0.014668,0.009223,0.000908
115.00,86.28,18.0,135,48.150,0.962105,-0.066050,0.032152,0.002451,65.23,12.0,1265,0.100,-0.011010,-0.017551,0.011300,0.001140
117.50,71.88,2.0,421,45.375,0.975503,-0.038340,0.022403,0.002050,63.87,45.0,1005,0.130,-0.014078,-0.021261,0.013981,0.001440
120.00,59.86,18.0,456,42.725,0.985577,-0.020346,0.014275,0.001569,62.79,317.0,6089,0.170,-0.018288,-0.026155,0.017495,0.001833


In [6]:
info1

('NVDA',
 datetime.datetime(2022, 8, 26, 0, 0),
 datetime.datetime(2022, 9, 16, 0, 0),
 162.5517)

### Gamma exposure function

In [7]:
def getgex(dfchain, spotprice, ptfactor=100, decplace=4):
    """Obtain gamma exposure of each option contract per 1% move of underlying."""
    dfgex = dfchain.copy()
    dfgex['c_GEX'] = dfgex['c_gamma'] * dfgex['c_oi'] * ptfactor * spotprice / 100
    dfgex['p_GEX'] = -dfgex['p_gamma'] * dfgex['p_oi'] * ptfactor * spotprice / 100
    dfgex['GEX'] = dfgex['c_GEX'] + dfgex['p_GEX']
    for col in ['c_GEX', 'p_GEX', 'GEX']:
        dfgex[col] = np.round(dfgex[col], decplace)
    
    return dfgex

In [8]:
dfgex1 = getgex(dfop1, info1[-1])
dfgex1[(dfgex1['c_mid'] > 1) & (dfgex1['p_mid'] > 1)]

,c_iv,c_vol,c_oi,c_mid,c_delta,c_theta,c_vega,c_gamma,p_iv,p_vol,p_oi,p_mid,p_delta,p_theta,p_vega,p_gamma,c_GEX,p_GEX,GEX
strike,,,,,,,,,,,,,,,,,,,
140.0,61.77,41.0,867,24.425,0.860399,-0.127385,0.086615,0.009224,57.32,4115.0,15398,1.455,-0.124038,-0.108949,0.079830,0.009161,1299.9138,-22930.1435,-21630.2297
145.0,54.47,148.0,856,19.700,0.826359,-0.129704,0.100010,0.012078,56.16,2131.0,7042,2.270,-0.179943,-0.136776,0.102290,0.011981,1680.5210,-13714.6045,-12034.0835
150.0,55.95,214.0,3466,16.125,0.747263,-0.166007,0.124617,0.014651,54.94,6643.0,17923,3.400,-0.249616,-0.161943,0.123801,0.014823,8254.4303,-43184.3750,-34929.9447
155.0,54.86,131.0,2521,12.675,0.665422,-0.185448,0.141976,0.017024,53.63,1853.0,6946,4.900,-0.332100,-0.180759,0.141560,0.017363,6976.1329,-19604.2253,-12628.0923
157.5,53.91,258.0,315,11.050,0.621264,-0.190361,0.148306,0.018096,53.24,884.0,553,5.850,-0.377873,-0.187863,0.148201,0.018311,926.5764,-1645.9691,-719.3928
160.0,53.38,382.0,10004,9.600,0.574402,-0.194246,0.152835,0.018834,52.41,5331.0,19356,6.850,-0.425157,-0.190676,0.152803,0.019178,30626.7047,-60341.3148,-29714.6101
162.5,52.75,391.0,237,8.250,0.526223,-0.194939,0.155212,0.019355,51.78,650.0,569,8.000,-0.474222,-0.191368,0.155223,0.019719,745.6469,-1823.8502,-1078.2033
165.0,51.99,2415.0,9247,7.000,0.477063,-0.192228,0.155291,0.019648,51.36,3723.0,11458,9.300,-0.523823,-0.189874,0.155271,0.019886,29533.1185,-37038.6687,-7505.5502
167.5,51.59,724.0,200,5.925,0.428397,-0.187979,0.153036,0.019513,50.77,3023.0,1990,10.700,-0.573525,-0.184826,0.152899,0.019810,634.3660,-6408.1451,-5773.7790


### Visualize GEX

In [9]:
def gettotalgex(dfgex):
    """Obtain call-GEX, put-GEX and total-GEX."""
    return round(dfgex['c_GEX'].sum(), 2), round(dfgex['p_GEX'].sum(), 2), round(dfgex['GEX'].sum(), 2)

def plotgex(dfgex, info):
    """Plot call GEX, put GEX and net GEX."""
    tdstr = info[1].strftime('%Y-%m-%d')
    expiry = info[2].strftime('%Y-%m-%d')
    spotprice = info[-1]
    gexsum_c, gexsum_p, gexsum = gettotalgex(dfgex)
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.125, row_heights=[8.0, 8.0],
                        specs=[[{"type": "bar"}], [{"type": "bar"}]], 
                        subplot_titles=(f'Call-GEX: {gexsum_c}, Put-GEX: {gexsum_p}', f'Net GEX: {gexsum}'))
        
    fig.add_trace(go.Bar(x=dfgex.index, y=dfgex['c_GEX'], name="Call GEX",
                         marker=dict(color='#34dd74', line=dict(color='#34dd74', width=2))), 
                  row=1, col=1)
    
    fig.add_trace(go.Bar(x=dfgex.index, y=dfgex['p_GEX'], name="Put GEX",
                         marker=dict(color='#dd3462', line=dict(color='#dd3462', width=2))), 
                  row=1, col=1)
    
    fig.add_trace(go.Bar(x=dfgex.index, y=dfgex['GEX'], name="GEX",
                         marker=dict(color='#1956ee', line=dict(color='#1956ee', width=2))), 
                  row=2, col=1)
    
    titletext = f'{info[0]} GEX of expiry {expiry} on {tdstr} at {spotprice} \n'
    
    fig.update_layout(title=titletext, title_x=0.5, width=1000, height=800)
    fig.show()

In [10]:
plotgex(dfgex1, info1)

### Compute GEX for general spot price 

In [11]:
def getnewgex(dfgex, info, spotprice=180, ptfactor=100, decplace=4):
    """Obtain total GEX of an option chain with arbitary underlying price."""
    # Alter underlying price
    dfnewgex = dfgex.copy()
    # Recompute option price, delta and gamma
    dfnewgex.reset_index(inplace=True)
    tdays = (info[2] - info[1]).days
    colbs_c = dfnewgex.apply(lambda row: BSModel(spotprice, row['strike'], tdays, row['c_iv'] / 100), axis=1)
    dfnewgex['c_close'] = np.round(colbs_c.apply(lambda x: x.cprice), 2)
    dfnewgex['c_delta'] = np.round(colbs_c.apply(lambda x: x.cdelta), 4)
    dfnewgex['c_gamma'] = colbs_c.apply(lambda x: x.gamma)
    colbs_p = dfnewgex.apply(lambda row: BSModel(spotprice, row['strike'], tdays, row['p_iv'] / 100), axis=1)
    dfnewgex['p_close'] = np.round(colbs_p.apply(lambda x: x.pprice), 2)
    dfnewgex['p_delta'] = np.round(colbs_p.apply(lambda x: x.pdelta), 4)
    dfnewgex['p_gamma'] = colbs_p.apply(lambda x: x.gamma)    
    # GEX
    dfnewgex['c_GEX'] = np.round(dfnewgex['c_gamma'] * dfnewgex['c_oi'] * ptfactor * spotprice / 100, decplace) 
    dfnewgex['p_GEX'] = np.round(dfnewgex['p_gamma'] * dfnewgex['p_oi'] * ptfactor * -1 * spotprice / 100, decplace)
    dfnewgex['GEX'] = np.round(dfnewgex['c_GEX'] + dfnewgex['p_GEX'], decplace)
    # Set back `strike` as index
    dfnewgex.set_index('strike', inplace=True)
    
    return dfnewgex

### Zero Gamma Level

In [12]:
def get0gamma(dfchain, info, ptfactor=100, rangefactor=0.125, gridfactor=0.0125):
    """Obtain zero gamma level."""
    # GEX according to true underlying price
    spotprice = info[-1]
    dfgex = getgex(dfchain, spotprice)
    cgex0, pgex0, ngex0 = gettotalgex(dfgex)
    # Grid of spot price levels (in geometric sequence)
    exprange = np.arange(-rangefactor, rangefactor, gridfactor)
    spotlevels = [spotprice * np.exp(exp) for exp in exprange]

    # Obtain GEX for all spot price levels
    dfsumgex = pd.DataFrame()
    for level in spotlevels:
        dfnewgex = getnewgex(dfgex, info, level, ptfactor)
        cgex, pgex, ngex = gettotalgex(dfnewgex)
        dfsumgex.loc[level, 'c_GEX'] = cgex
        dfsumgex.loc[level, 'p_GEX'] = pgex
        dfsumgex.loc[level, 'GEX'] = ngex
    # Obtain zero gamma level by linear interpolation
    if (dfsumgex['GEX'].min() * dfsumgex['GEX'].max()) > 0:
        zerogexlevel = None
    else:
        for p1, p2 in zip(dfsumgex.index[:-1], dfsumgex.index[1:]):
            level1 = dfsumgex.loc[p1, 'GEX']
            level2 = dfsumgex.loc[p2, 'GEX']
            if level1 * level2 < 0:
                zerogexlevel = round((p1 * level2 - p2 * level1) / (level2 - level1), 2)
                break
    # Visualize GEX at different spot price 
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, row_heights=[6.0], specs=[[{"type": "scatter"}]])
    
    fig.add_trace(go.Scatter(x=dfsumgex.index, y=dfsumgex['GEX'], mode='lines+markers', name='spot levels'), 
                  row=1, col=1)
    
    fig.add_trace(go.Scatter(x=[spotprice], y=[ngex0], mode='markers', name='spot price', 
                             marker=dict(size=20, color='#66dc19')), row=1, col=1)
    
    if zerogexlevel != None:
        fig.add_trace(go.Scatter(x=[zerogexlevel], y=[0], mode='markers', name='0-gamma level', 
                                 marker=dict(size=20, color=' #ee195a')), row=1, col=1)
        
    tdstr = info[1].strftime('%Y-%m-%d')
    expiry = info[2].strftime('%Y-%m-%d')
    fig.update_layout(title=f'Gamma levels of {info[0]} of expiry {expiry} on {tdstr}', 
                      title_x=0.5, width=1000, height=800)
    fig.show()

    return dfsumgex, zerogexlevel

In [13]:
dfsumgex1, zerogexlevel0 = get0gamma(dfop1, info1, rangefactor=0.2)

..\bsoption\bsmodel.py:37: RuntimeWarning:

divide by zero encountered in double_scalars

..\bsoption\bsmodel.py:33: RuntimeWarning:

invalid value encountered in double_scalars

